In [ ]:
pip install retry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
#WITH MODIFIED SNOW QA, USING MCD ALBEDO
import ee
import logging
import multiprocessing
from retry import retry
import sys
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

ee.Authenticate()
ee.Initialize(project='ee-cooleysarahw')
#ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

#get input data
table = ee.FeatureCollection('projects/ee-cooleysarahw/assets/grids_lakes_4_dec17')
lakes = ee.FeatureCollection('projects/ee-cooleysarahw/assets/pld_grid4_forlakes_v4')
lakes_buff = ee.FeatureCollection('projects/ee-cooleysarahw/assets/pld_grid4_forland_v4')

#lakes = ee.FeatureCollection('projects/ee-cooleysarahw/assets/grid2_forlakes_v2')
#lakes_buff = ee.FeatureCollection('projects/ee-cooleysarahw/assets/grid2_forland_v2')

## MAIN

@retry(tries=5, delay=10,backoff=2)
def getResult(index,gridID):

    #GET ROIS
    rois = ee.FeatureCollection(table).sort('id');
    grid = ee.Feature(rois.filter(ee.Filter.eq('id',gridID)).first());
    roi = ee.Geometry.MultiPolygon(grid.geometry().getInfo()['coordinates'][:])

    #CREATE LAKE AND LAND MASKS
    dataset = ee.ImageCollection('MODIS/006/MOD44W').filter(ee.Filter.date('2010-01-01','2010-10-01'))
    dataset = dataset.first().clip(roi)
    mask = dataset.select('water_mask').gte(0);
    lake_mask = mask.clipToCollection(lakes)
    lake_maskland = mask.clipToCollection(lakes_buff)
    l_mask = lake_maskland.unmask();
    land_mask = mask.subtract(l_mask);

    #CLIP IMAGE
    def clip_image(image):
       return(image.clip(roi))

    #BITWISE EXTRACT
    def bitwiseExtract(value, fromBit, toBit):
        maskSize = ee.Number(1).add(toBit).subtract(fromBit)
        mask = ee.Number(1).leftShift(maskSize).subtract(1)
        return value.rightShift(fromBit).bitwiseAnd(mask)

    #MASK MODIS CLOUDS
    def maskClouds(image):
        qa = image.select('state_1km')
        cloud1 = bitwiseExtract(qa, 0, 0)
        cloud2 = bitwiseExtract(qa, 1, 1)
        mask1 = cloud1.eq(0).And(cloud2.eq(1))
        mask2 = cloud1.eq(1).And(cloud2.eq(0))
        zenith = image.select('SolarZenith')
        zmask = zenith.gte(8000)
        cloud_mask = mask1.add(mask2).add(zmask)
        cloud_mask = cloud_mask.gte(1).rename('cloud_mask');
        return image.addBands([cloud_mask])

    #GET SNOW AND ALBEDO
    def get_snow_and_albedo(image):
        b02 = image.select('sur_refl_b02')
        cloud_mask = image.select('cloud_mask')
        snow = image.select('NDSI_Snow_Cover').updateMask(image.select('cloud_mask').eq(0));
        snowqa = image.select('NDSI_Snow_Cover_Basic_QA')
        classified_snow1 = (snow.gte(snowthresh).And(snow.lte(100))).And(snowqa.lt(2))
        classified_snow2 = snowqa.gte(2).And(b02.gte(b2snow))
        classified_snow = classified_snow1.add(classified_snow2)
        classified_snow = classified_snow.gte(1).rename('classified_snow')
        classified_water1 = snow.lte(snowthresh).And(snowqa.lt(2))
        classified_water2 = snowqa.gte(2).And(b02.lte(b2land))
        classified_water = classified_water1.add(classified_water2)
        classified_water = classified_water.gte(1).rename('classified_water')
        cloud_mask = cloud_mask.add(snowqa.gte(2).And(b02.gt(b2land)).And(b02.lt(b2snow)))
        cloud_mask = cloud_mask.gte(1)
        quality_mask = image.select('BRDF_Albedo_Band_Mandatory_Quality_shortwave')
        quality_mask = bitwiseExtract(quality_mask,0,0).eq(0);
        snow_mask = classified_snow.add(quality_mask)
        snow_mask = snow_mask.gte(1)
        water_mask = classified_water.add(quality_mask)
        water_mask = water_mask.gte(1)
        water_albedo = image.select('Albedo_WSA_shortwave').updateMask(water_mask).rename('water_albedo')
        snow_albedo = image.select('Albedo_WSA_shortwave').updateMask(snow_mask).rename('snow_albedo')
        return image.addBands([water_albedo,snow_albedo,classified_water,classified_snow])

    #GET VALUES FOR LAND
    def get_land_values(image):
      im = image.updateMask(land_mask)
      water_out = im.select('classified_water').reduceRegion(
          reducer = ee.Reducer.sum(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'classified_water': -999}, False).getNumber('classified_water').format('%.2f')
      snow_out = im.select('classified_snow').reduceRegion(
          reducer = ee.Reducer.sum(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'classified_snow': -999}, False).getNumber('classified_snow').format('%.2f')
      cloud_out = im.select('cloud_mask').reduceRegion(
          reducer = ee.Reducer.sum(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'cloud_mask': -999}, False).getNumber('cloud_mask').format('%.2f')
      snow_alb = im.select('snow_albedo').reduceRegion(
          reducer = ee.Reducer.mean(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'snow_albedo': -999}, False).getNumber('snow_albedo').format('%.2f')
      water_alb = im.select('water_albedo').reduceRegion(
          reducer = ee.Reducer.mean(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'water_albedo': -999}, False).getNumber('water_albedo').format('%.2f')
      snow_std = im.select('snow_albedo').reduceRegion(
          reducer = ee.Reducer.stdDev(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'snow_albedo': -999}, False).getNumber('snow_albedo').format('%.2f')
      water_std = im.select('water_albedo').reduceRegion(
          reducer = ee.Reducer.stdDev(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'water_albedo': -999}, False).getNumber('water_albedo').format('%.2f')
      date = image.date().format('yyyy-MM-dd')
      return image.set('land_output',[cloud_out,snow_out,water_out,snow_alb,water_alb,snow_std,water_std,date])

    #GET VALUES FOR Water
    def get_water_values(image):
      im = image.updateMask(lake_mask)
      water_out = im.select('classified_water').reduceRegion(
          reducer = ee.Reducer.sum(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'classified_water': -999}, False).getNumber('classified_water').format('%.2f')
      snow_out = im.select('classified_snow').reduceRegion(
          reducer = ee.Reducer.sum(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'classified_snow': -999}, False).getNumber('classified_snow').format('%.2f')
      cloud_out = im.select('cloud_mask').reduceRegion(
          reducer = ee.Reducer.sum(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'cloud_mask': -999}, False).getNumber('cloud_mask').format('%.2f')
      snow_alb = im.select('snow_albedo').reduceRegion(
          reducer = ee.Reducer.mean(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'snow_albedo': -999}, False).getNumber('snow_albedo').format('%.2f')
      water_alb = im.select('water_albedo').reduceRegion(
          reducer = ee.Reducer.mean(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'water_albedo': -999}, False).getNumber('water_albedo').format('%.2f')
      snow_std = im.select('snow_albedo').reduceRegion(
          reducer = ee.Reducer.stdDev(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'snow_albedo': -999}, False).getNumber('snow_albedo').format('%.2f')
      water_std = im.select('water_albedo').reduceRegion(
          reducer = ee.Reducer.stdDev(),
          geometry = image.geometry(),
          scale = 500
          ).combine({'water_albedo': -999}, False).getNumber('water_albedo').format('%.2f')
      date = image.date().format('yyyy-MM-dd')
      return image.set('water_output',[cloud_out,snow_out,water_out,snow_alb,water_alb,snow_std,water_std,date])

    #MAIN SCRIPT
    mod09ga = ee.ImageCollection('MODIS/061/MOD09GA').filterBounds(roi).filter(ee.Filter.date(s1, s2)).select(['state_1km','SolarZenith','sur_refl_b02'])
    mcd43a3 = ee.ImageCollection('MODIS/061/MCD43A3').filterBounds(roi).filter(ee.Filter.date(s1, s2)).select(['Albedo_WSA_shortwave','BRDF_Albedo_Band_Mandatory_Quality_shortwave'])
    mod10a1 = ee.ImageCollection('MODIS/061/MOD10A1').filterBounds(roi).filter(ee.Filter.date(s1, s2)).select(['NDSI_Snow_Cover','NDSI_Snow_Cover_Basic_QA'])

    mod_merged1 = mod09ga.combine(mod10a1)
    mod_merged2 = mod_merged1.combine(mcd43a3)
    mod_clipped = mod_merged2.map(clip_image)
    masked = mod_clipped.map(maskClouds)
    classified = masked.map(get_snow_and_albedo)
    landadded = classified.map(get_land_values)
    wateradded = landadded.map(get_water_values)

    #AGGREGATE AND WRITE LAKES
    result_lakes = wateradded.aggregate_array('water_output').getInfo()
    filename = fname+ 'lakes_' +str(gridID)+'.csv'
    directory = '/content/drive/MyDrive/Research/Lake_Ice_Albedo_project/0_time_series_v2/grid4_updates/'
    with open(directory+filename, 'w') as out_file:
        for items in result_lakes:
            line = ','.join([str(item) for item in items])
            print(line, file=out_file)
    print("Done: "+filename, flush=True)

    #AGGREGATE AND WRITE LAND
    result_land = landadded.aggregate_array('land_output').getInfo()
    filename = fname+ 'land_' +str(gridID)+'.csv'
    with open(directory+filename, 'w') as out_file:
        for items in result_land:
            line = ','.join([str(item) for item in items])
            print(line, file=out_file)
    print("Done: "+filename, flush=True)


if __name__ == '__main__':

  # set up flush for logging
  class FlushStreamHandler(logging.StreamHandler):
    def emit(self, record):
        super().emit(record)
        self.flush()

  # Set up logging using basicConfig, using custom handler to ensure that prints flush to the console
  logging.basicConfig(
      level=logging.DEBUG,
      format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Set the format
      handlers=[FlushStreamHandler(sys.stderr)]
  )

  gridIDs = [1004, 1005, 1042, 1043, 1078, 1079, 1113, 1114, 1147, 1148]

  startList = ['2001-01-01','2002-01-01','2003-01-01','2004-01-01','2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01','2011-01-01','2012-01-01','2013-01-01','2014-01-01','2015-01-01','2016-01-01','2017-01-01','2018-01-01','2019-01-01','2020-01-01','2021-01-01','2022-01-01']
  endList = ['2002-01-01','2003-01-01','2004-01-01','2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01','2011-01-01','2012-01-01','2013-01-01','2014-01-01','2015-01-01','2016-01-01','2017-01-01','2018-01-01','2019-01-01','2020-01-01','2021-01-01','2022-01-01','2023-01-01']
  nameList = ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']

  snowthresh = 50
  b2snow = 5000
  b2land = 2000
  n = len(startList)
  for i in range(20,22):
    fname = 'grid4_'+nameList[i] +'_'
    s1 = startList[i];
    s2 = endList[i];
    pool = multiprocessing.Pool(30)
    pool.starmap(getResult, enumerate(gridIDs))
    pool.close()
    pool.join()
    print('Finished'+nameList[i])


Mounted at /content/drive


In [ ]:
gridIDs = list(range(214,751))
print(gridIDs)


[214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413,

In [ ]:
import numpy as np

In [ ]:
#check for files
from pathlib import Path
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# List of file paths relative to your Google Drive root
gridIDs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750]



file_status = []
# Check if each file exists
for i in range(len(gridIDs)):
    filename = '/content/drive/MyDrive/Research/Lake_Ice_Albedo_project/0_time_series_v2/grid1_2001/' + 'grid1_2021_land_' + str(gridIDs[i])+'.csv'
    file_path = Path(filename)
    if file_path.exists():
        file_status.append(1)  # Append 1 if the file exists
    else:
        file_status.append(0)  # Append 0 if the file does not exist


output_IDs = [gridIDs[i] for i in range(len(file_status)) if file_status[i] == 0]
print(output_IDs)
print(len(output_IDs))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[]
0
